# Imports and stuff

In [60]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [61]:
import pandas as pd
import allel

from scripts import utils, stat, genom_pca

In [62]:
VCF_FILE_PATH = r'..\data\genotypes.vcf'
OUTPUT_CSV_FILE_PATH = r'..\data\parsed_vcf.csv'
H5_FILE_PATH= r'..\data\genotypes.h5'
WEATHER_DATA_PATH = r'..\data\weather_by_year_v2.csv'

# Статистические данные

In [63]:
utils.parse_vcf_2_csv(VCF_FILE_PATH, OUTPUT_CSV_FILE_PATH)

Parsed VCF data saved to ..\data\parsed_vcf.csv


In [64]:
df = pd.read_csv(r'..\data\parsed_vcf.csv')
df.head(2)

C:\Users\AlexTanya\AppData\Local\Temp\ipykernel_25616\2392676959.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'..\data\parsed_vcf.csv')


,Sample,Chromosome,Position,Reference,Alternate,INFO,Genotype,List of Phred-scaled genotype likelihoods,Raw read depth
0,PS000026,1,40481,G,A,"dict_keys(['GT', 'PL', 'DP'])",0/0,"0,3,39",1
1,PS000027,1,40481,G,A,"dict_keys(['GT', 'PL', 'DP'])",0/0,"0,9,89",3


## Получение статистических признаков

#### 1. **Инициализация функции:**

- Функция `extract_comprehensive_genomic_features` принимает на вход DataFrame `df`, содержащий геномные данные.

- Геномные данные содержат колонки, такие как:

  - `Sample`: идентификатор образца.

  - `Genotype`: генотип варианта (`0/0`, `0/1`, `1/1`, и др.).

  - `Chromosome`: хромосома, на которой находится вариант.

  - `Position`: позиция варианта на хромосоме.

  - `Reference`, `Alternate`: аллели, которые были обнаружены.

  - `Raw read depth`: глубина чтения.

  - `List of Phred-scaled genotype likelihoods`: вероятности генотипа в формате Phred.



---



#### 2. **Обработка каждого образца:**

- Для каждого образца в данных применяется функция `advanced_sample_analysis`, которая анализирует множество характеристик.



---



#### 3. **Извлекаемые признаки:**



1. **Генотипический анализ:**

   - `total_variants`: общее число вариантов.

   - Подсчёт частоты различных генотипов, таких как `0/0`, `1/1`, `./.`, и т.д.



2. **Хромосомный профиль:**

   - Количество и доля вариантов для каждой хромосомы (`chromosome_{chrom}_variant_count` и `chromosome_{chrom}_variant_ratio`).



3. **Позиционный анализ:**

   - Энтропия распределения позиций (`position_entropy`).

   - Статистики по позициям: медиана, дисперсия, размах и плотность.



4. **Аллельный анализ:**

   - Количество уникальных аллелей (`allele_diversity`).

   - Соотношение референсных и альтернативных аллелей (`ref_to_alt_ratio`).

   - Подсчёт числа аллелей типа `A`, `T`, `G`, `C` в референсных и альтернативных данных.



5. **Глубина чтения:**

   - Среднее, медиана, стандартное отклонение и квартильные значения для глубины чтения (`read_depth_mean`, `read_depth_median`, и т.д.).



6. **Анализ вероятностей генотипов (Phred-значения):**

   - Среднее, максимум и дисперсия вероятностей генотипа (`pl_mean`, `pl_max`, `pl_variance`).



7. **Транзитивный анализ:**

   - Подсчёт переходов между базами, например: `A>G`, `C>T` (`transition_AG_count`, `transition_CT_count`).



8. **Распределение типов вариантов:**

   - Частотный анализ преобразований между референсными и альтернативными аллелями (`variant_type_1`, `variant_type_2`, ...).



---



#### 4. **Возвращение результата:**

- Все вычисленные признаки агрегируются для каждого образца и возвращаются в виде DataFrame, где каждая строка соответствует отдельному образцу, а столбцы — извлечённым признакам.


In [65]:
result = stat.extract_comprehensive_genomic_features(df)

In [66]:
result.head(2)

,Sample,total_variants,0/0_homozygous_count,0/1_homozygous_count,1/1_homozygous_count,./._homozygous_count,1/2_homozygous_count,0/2_homozygous_count,2/2_homozygous_count,1/3_homozygous_count,...,variant_type_2,variant_type_3,variant_type_4,variant_type_5,variant_type_6,variant_type_7,variant_type_8,variant_type_9,variant_type_10,variant_type_11
0,PS000026,38940.0,16814.0,4712.0,7452.0,9898.0,32.0,26.0,2.0,2.0,...,0.165742,0.131895,0.131844,0.046687,0.046097,0.045378,0.042938,0.035465,0.034155,0.031253
1,PS000027,38940.0,21268.0,5224.0,9381.0,2985.0,57.0,15.0,8.0,0.0,...,0.165742,0.131895,0.131844,0.046687,0.046097,0.045378,0.042938,0.035465,0.034155,0.031253


## Удалим статичные столбцы

In [67]:
static_columns = [col for col in result.columns if result[col].nunique() == 1]

In [68]:
len(static_columns)

250

In [69]:
new_result = result.drop(columns=static_columns, axis=1)

## Теперь преобразуем .vcf в .h5

In [70]:
allel.vcf_to_hdf5(
    input=VCF_FILE_PATH,
    output=H5_FILE_PATH,
    overwrite=True
)

e:\AgroCode\.venv\Lib\site-packages\allel\io\vcf_read.py:1732: UserWarning: invalid INFO header: '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">\n'
  warnings.warn('invalid INFO header: %r' % header)


## Далее работаем с .h5 файлом, используем идеи из [статьи](https://link.springer.com/content/pdf/10.1007/s00122-024-04687-w.pdf)

In [71]:
LD_PRUNE_DICT = dict(size=100, step=20, threshold=.1, n_iter=1)
PCA_PARAMETERS_DICT = dict(n_components=15, scaler='patterson')

In [72]:
chromo_dict = genom_pca.parse_genotype_by_chromo(H5_FILE_PATH)
feature_dict = genom_pca.get_pca_vectors(chromo_dict, PCA_PARAMETERS_DICT, False, True, LD_PRUNE_DICT)

iteration 1 retaining 241 removing 1239 variants
iteration 1 retaining 287 removing 1883 variants
iteration 1 retaining 162 removing 825 variants
iteration 1 retaining 141 removing 872 variants
iteration 1 retaining 219 removing 1664 variants
iteration 1 retaining 255 removing 1695 variants
iteration 1 retaining 276 removing 2287 variants
iteration 1 retaining 205 removing 1731 variants
iteration 1 retaining 231 removing 1462 variants
iteration 1 retaining 373 removing 2332 variants
iteration 1 retaining 261 removing 2013 variants
iteration 1 retaining 251 removing 1527 variants
iteration 1 retaining 239 removing 1461 variants
iteration 1 retaining 256 removing 1800 variants
iteration 1 retaining 292 removing 2375 variants
iteration 1 retaining 228 removing 1296 variants
iteration 1 retaining 258 removing 1846 variants
iteration 1 retaining 230 removing 1273 variants
iteration 1 retaining 216 removing 1522 variants
iteration 1 retaining 299 removing 1774 variants
iteration 1 retaining 

In [73]:
df_list = []
for key, value in feature_dict.items():
    feature_vector = value['feature_vector']
    
    # Create a DataFrame for each key where each column corresponds to a sample
    sample_df = pd.DataFrame({
        f"Sample_{i+1}": [feature_vector[i]] for i in range(feature_vector.shape[0])
    })

    sample_df['Chroma'] = key
    
    df_list.append(sample_df)

# Concatenate all DataFrames into a single DataFrame
PCA_df = pd.concat(df_list, ignore_index=True)

PCA_df = PCA_df.T
PCA_df.columns = PCA_df.iloc[-1]
PCA_df.to_csv(r'../data/20_chromas+KZ.csv')

In [74]:
PCA_df = PCA_df.reset_index().drop('index', axis=1)

In [75]:
new_result_w_pca20 = pd.concat([new_result, PCA_df], axis=1)
new_result_w_pca20.head()

,Sample,0/0_homozygous_count,0/1_homozygous_count,1/1_homozygous_count,./._homozygous_count,1/2_homozygous_count,0/2_homozygous_count,2/2_homozygous_count,1/3_homozygous_count,2/3_homozygous_count,...,2,20,3,4,5,6,7,8,9,KZ
0,PS000026,16814.0,4712.0,7452.0,9898.0,32.0,26.0,2.0,2.0,1.0,...,"[-5.784528, 0.4498071, -0.017342787, 5.2753015...","[-6.8180027, 0.48695463, 3.5135922, 1.5036408,...","[-0.57276464, -2.44983, 8.653072, 0.19224861, ...","[-8.527061, 0.3257611, 5.834775, -9.110653, -4...","[-4.185814, 0.5078453, -1.3162014, 2.3343368, ...","[-1.9142941, 3.8798106, -4.7131076, -5.2503343...","[-1.6162633, 6.399207, 1.2315081, -4.7762446, ...","[-2.9762053, 3.760658, -2.5448658, -0.04256083...","[-1.4386533, -4.25053, -6.1608253, -1.1426252,...","[-7.383055, 5.321986, -2.789504, -1.041765, 0...."
1,PS000027,21268.0,5224.0,9381.0,2985.0,57.0,15.0,8.0,0.0,2.0,...,"[0.47154856, -2.7073226, -3.6026685, 1.8215551...","[-5.659049, 1.9701648, -2.1127121, 14.879212, ...","[-6.1576257, 10.036709, -1.6141998, -1.6323696...","[-12.797119, 1.7392356, 2.9041936, 3.5752747, ...","[-4.8123093, 9.927958, 5.9969397, 14.8058605, ...","[0.5490302, 2.106615, -1.2816659, 1.209668, 0....","[0.1745142, 0.6913679, 5.5662436, -6.0029535, ...","[3.9509366, 1.3792801, -9.295904, -1.2021961, ...","[3.6313097, 0.65917706, 6.95152, -1.0582379, 7...","[-1.3708509, -0.15660581, 5.8599405, -4.236762..."
2,PS000028,18585.0,13014.0,6680.0,525.0,79.0,48.0,8.0,0.0,0.0,...,"[3.716692, -1.8984091, 6.9937725, -0.83207273,...","[7.8131375, -3.1270459, 3.9723756, -1.3064128,...","[1.2198064, 0.05057852, -2.5421603, 2.4653072,...","[0.24978212, -2.7319953, 0.7617265, 1.276161, ...","[7.0305324, -4.814624, 0.7927814, -0.41272318,...","[1.9677361, 0.4134413, 1.7326355, 0.066696964,...","[4.1076446, -7.1776567, 1.9098437, -3.1995747,...","[-5.676147, -2.228611, -3.648197, 0.93701094, ...","[2.9668674, 1.0013124, -0.96189415, 1.4935846,...","[5.9869947, 1.0101153, -4.5697, 5.2848716, -5...."
3,PS000031,21261.0,5468.0,11161.0,933.0,77.0,29.0,11.0,0.0,0.0,...,"[3.9231558, -9.248388, 7.851115, 5.7313995, -3...","[2.360415, -1.5286882, 4.3701696, -2.9207542, ...","[1.075702, 13.57204, -4.378609, 8.742799, -4.3...","[7.552921, -8.502733, 5.8556657, 4.9472985, -4...","[-0.12174382, 4.0717864, 1.2505251, -4.8692074...","[7.7382565, -4.103768, -6.3458323, 0.8121979, ...","[9.469856, 2.9780326, 5.19628, -1.0091786, -0....","[-5.9287267, -9.974031, 3.2413785, -0.30961785...","[5.5596952, 4.2486978, -1.0353271, 6.1300297, ...","[6.749848, 0.10785126, -0.6354936, -1.9671026,..."
4,PS000033,21861.0,5077.0,11414.0,456.0,82.0,33.0,14.0,2.0,1.0,...,"[10.71392, -2.4432747, -2.4565356, 3.0316508, ...","[9.120246, 7.2733335, -2.926389, -1.88119, -5....","[13.8542795, 1.7871468, -8.50231, -8.674371, 7...","[-14.811771, 2.9275856, -4.62993, 10.26322, -3...","[10.20709, -1.0420249, 1.0421253, 1.6366296, 1...","[7.84602, -2.9849668, -0.5951567, -0.6921777, ...","[2.709554, -5.1540413, 5.917786, -1.477546, 2....","[0.9930004, -5.2809286, -12.700347, -7.0996714...","[9.656525, 3.950626, 3.3923395, -2.513554, -1....","[5.0899525, -1.823737, 0.4056462, 3.96363, -2...."


In [76]:
ph = pd.read_csv(r'../data/phenotypes.tsv', sep='\t')
vg = pd.read_csv(r'../data/vegetation.tsv', sep='\t')

In [77]:
ph = ph.rename(columns={'sample': 'Sample'})
vg = vg.rename(columns={'sample': 'Sample'})

In [78]:
vg_w_ph = pd.merge(vg, ph, right_on='Sample', left_on='Sample')

In [79]:
train = pd.merge(new_result_w_pca20, vg_w_ph, right_on='Sample', left_on='Sample')

# Подготовим датафрейм для обучения

In [80]:
id_vars = ['Sample', '0/0_homozygous_count', '0/1_homozygous_count',
       '1/1_homozygous_count', './._homozygous_count', '1/2_homozygous_count',
       '0/2_homozygous_count', '2/2_homozygous_count', '1/3_homozygous_count',
       '2/3_homozygous_count', '0/3_homozygous_count', '3/3_homozygous_count',
       'chromosome_20_variant_count', 'chromosome_20_variant_ratio',
       'read_depth_mean', 'read_depth_median', 'read_depth_std',
       'read_depth_quartile1', 'read_depth_quartile3', 'pl_mean',
       'pl_variance', '1', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '2', '20', '3', '4', '5', '6', '7', '8', '9', 'vegetation']
year_columns = ['2015', '2016',	'2017', '2019', '2020', '2021', '2022', '2023']

df_melted = train.melt(id_vars=id_vars, value_vars=year_columns, 
                    var_name='year', value_name='target')

In [81]:
df_melted.head()

,Sample,0/0_homozygous_count,0/1_homozygous_count,1/1_homozygous_count,./._homozygous_count,1/2_homozygous_count,0/2_homozygous_count,2/2_homozygous_count,1/3_homozygous_count,2/3_homozygous_count,...,3,4,5,6,7,8,9,vegetation,year,target
0,PS000026,16814.0,4712.0,7452.0,9898.0,32.0,26.0,2.0,2.0,1.0,...,"[-0.57276464, -2.44983, 8.653072, 0.19224861, ...","[-8.527061, 0.3257611, 5.834775, -9.110653, -4...","[-4.185814, 0.5078453, -1.3162014, 2.3343368, ...","[-1.9142941, 3.8798106, -4.7131076, -5.2503343...","[-1.6162633, 6.399207, 1.2315081, -4.7762446, ...","[-2.9762053, 3.760658, -2.5448658, -0.04256083...","[-1.4386533, -4.25053, -6.1608253, -1.1426252,...",110,2015,140.0
1,PS000027,21268.0,5224.0,9381.0,2985.0,57.0,15.0,8.0,0.0,2.0,...,"[-6.1576257, 10.036709, -1.6141998, -1.6323696...","[-12.797119, 1.7392356, 2.9041936, 3.5752747, ...","[-4.8123093, 9.927958, 5.9969397, 14.8058605, ...","[0.5490302, 2.106615, -1.2816659, 1.209668, 0....","[0.1745142, 0.6913679, 5.5662436, -6.0029535, ...","[3.9509366, 1.3792801, -9.295904, -1.2021961, ...","[3.6313097, 0.65917706, 6.95152, -1.0582379, 7...",113,2015,130.0
2,PS000028,18585.0,13014.0,6680.0,525.0,79.0,48.0,8.0,0.0,0.0,...,"[1.2198064, 0.05057852, -2.5421603, 2.4653072,...","[0.24978212, -2.7319953, 0.7617265, 1.276161, ...","[7.0305324, -4.814624, 0.7927814, -0.41272318,...","[1.9677361, 0.4134413, 1.7326355, 0.066696964,...","[4.1076446, -7.1776567, 1.9098437, -3.1995747,...","[-5.676147, -2.228611, -3.648197, 0.93701094, ...","[2.9668674, 1.0013124, -0.96189415, 1.4935846,...",114,2015,NaN
3,PS000031,21261.0,5468.0,11161.0,933.0,77.0,29.0,11.0,0.0,0.0,...,"[1.075702, 13.57204, -4.378609, 8.742799, -4.3...","[7.552921, -8.502733, 5.8556657, 4.9472985, -4...","[-0.12174382, 4.0717864, 1.2505251, -4.8692074...","[7.7382565, -4.103768, -6.3458323, 0.8121979, ...","[9.469856, 2.9780326, 5.19628, -1.0091786, -0....","[-5.9287267, -9.974031, 3.2413785, -0.30961785...","[5.5596952, 4.2486978, -1.0353271, 6.1300297, ...",114,2015,NaN
4,PS000033,21861.0,5077.0,11414.0,456.0,82.0,33.0,14.0,2.0,1.0,...,"[13.8542795, 1.7871468, -8.50231, -8.674371, 7...","[-14.811771, 2.9275856, -4.62993, 10.26322, -3...","[10.20709, -1.0420249, 1.0421253, 1.6366296, 1...","[7.84602, -2.9849668, -0.5951567, -0.6921777, ...","[2.709554, -5.1540413, 5.917786, -1.477546, 2....","[0.9930004, -5.2809286, -12.700347, -7.0996714...","[9.656525, 3.950626, 3.3923395, -2.513554, -1....",111,2015,116.0


In [82]:
df_filtered = df_melted.dropna(subset=['target']).reset_index(drop=True)
df_filtered['year'] = df_filtered['year'].astype(int)
df_filtered.shape

(359, 44)

# Добавим погодные данные

In [83]:
weather_features = pd.read_csv(WEATHER_DATA_PATH)
weather_features.head()

,year,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,daylight_duration,sunshine_duration,wind_speed_10m_max,...,diffuse_radiation_max,direct_normal_irradiance_max,global_tilted_irradiance_max,terrestrial_radiation_max,shortwave_radiation_instant_max,direct_radiation_instant_max,diffuse_radiation_instant_max,direct_normal_irradiance_instant_max,global_tilted_irradiance_instant_max,terrestrial_radiation_instant_max
0,2019,22.357153,12.248004,17.590923,21.007662,10.772452,16.315070,54583.350,40736.690,18.013605,...,417.0,885.83090,879.0,1160.9762,877.54956,768.72943,425.48828,885.83080,877.54956,1159.5826
1,2020,22.093102,12.040160,17.320866,20.570436,10.227975,15.777227,54526.100,40749.790,19.198004,...,381.0,883.62560,874.0,1161.0609,873.36530,760.05505,389.04240,883.62555,873.36530,1159.6711
2,2021,22.741140,12.801925,18.085735,21.437820,11.461225,16.889076,54545.240,40707.440,18.724500,...,386.0,924.62823,880.0,1161.1024,878.86340,770.00420,385.97424,924.62823,878.86340,1159.6475
3,2022,21.067938,11.707808,16.533870,19.626568,9.885121,15.026059,54563.290,38788.258,19.747036,...,384.0,924.06573,891.0,1161.0298,889.62100,783.78503,388.40158,924.06573,889.62100,1159.5903
4,2023,22.542774,12.048657,17.532032,21.157907,10.509547,16.172094,54581.645,40840.250,18.266440,...,418.0,897.78920,885.0,1161.0144,883.07580,774.31280,426.49698,897.78910,883.07580,1159.6368


In [84]:
train_with_weather = pd.merge(df_filtered, weather_features, on='year', how='left')
train_with_weather

,Sample,0/0_homozygous_count,0/1_homozygous_count,1/1_homozygous_count,./._homozygous_count,1/2_homozygous_count,0/2_homozygous_count,2/2_homozygous_count,1/3_homozygous_count,2/3_homozygous_count,...,diffuse_radiation_max,direct_normal_irradiance_max,global_tilted_irradiance_max,terrestrial_radiation_max,shortwave_radiation_instant_max,direct_radiation_instant_max,diffuse_radiation_instant_max,direct_normal_irradiance_instant_max,global_tilted_irradiance_instant_max,terrestrial_radiation_instant_max
0,PS000026,16814.0,4712.0,7452.0,9898.0,32.0,26.0,2.0,2.0,1.0,...,367.0,850.05743,862.0,1170.4559,868.3553,720.33624,368.08310,850.05743,868.3553,1170.5146
1,PS000027,21268.0,5224.0,9381.0,2985.0,57.0,15.0,8.0,0.0,2.0,...,367.0,850.05743,862.0,1170.4559,868.3553,720.33624,368.08310,850.05743,868.3553,1170.5146
2,PS000033,21861.0,5077.0,11414.0,456.0,82.0,33.0,14.0,2.0,1.0,...,367.0,850.05743,862.0,1170.4559,868.3553,720.33624,368.08310,850.05743,868.3553,1170.5146
3,PS000034,19640.0,5487.0,7561.0,6183.0,29.0,22.0,14.0,3.0,1.0,...,367.0,850.05743,862.0,1170.4559,868.3553,720.33624,368.08310,850.05743,868.3553,1170.5146
4,PS000048,20961.0,4954.0,12226.0,658.0,81.0,41.0,11.0,6.0,1.0,...,367.0,850.05743,862.0,1170.4559,868.3553,720.33624,368.08310,850.05743,868.3553,1170.5146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,PS000436,14383.0,4347.0,5722.0,14449.0,14.0,21.0,2.0,1.0,1.0,...,418.0,897.78920,885.0,1161.0144,883.0758,774.31280,426.49698,897.78910,883.0758,1159.6368
355,PS000440,20125.0,4766.0,8192.0,5794.0,32.0,24.0,6.0,1.0,0.0,...,418.0,897.78920,885.0,1161.0144,883.0758,774.31280,426.49698,897.78910,883.0758,1159.6368
356,PS000444,24313.0,3790.0,9522.0,1249.0,30.0,22.0,13.0,1.0,0.0,...,418.0,897.78920,885.0,1161.0144,883.0758,774.31280,426.49698,897.78910,883.0758,1159.6368
357,PS000568,24041.0,3967.0,10224.0,625.0,51.0,19.0,12.0,1.0,0.0,...,418.0,897.78920,885.0,1161.0144,883.0758,774.31280,426.49698,897.78910,883.0758,1159.6368


In [85]:
train_with_weather.to_csv(r'../data/train_file.csv', index=False)